In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from sklearn                import preprocessing 
from sklearn.metrics        import mean_squared_error,mean_absolute_percentage_error,mean_absolute_error
from sklearn.ensemble       import RandomForestRegressor
from sklearn.linear_model   import Lasso,Ridge

from sktime.forecasting.theta               import ThetaForecaster
from sktime.forecasting.fbprophet           import Prophet
from sktime.forecasting.arima               import ARIMA
from sktime.forecasting.base                import ForecastingHorizon
from sktime.forecasting.trend               import PolynomialTrendForecaster,TrendForecaster
from sktime.forecasting.exp_smoothing       import ExponentialSmoothing
from sktime.forecasting.statsforecast       import StatsForecastAutoARIMA
from sktime.forecasting.compose             import TransformedTargetForecaster,ForecastingPipeline
from sktime.transformations.series.detrend  import Deseasonalizer,Detrender
from sktime.forecasting.model_selection     import temporal_train_test_split
from sktime.forecasting.trend               import ProphetPiecewiseLinearTrendForecaster
from sktime.forecasting.compose             import make_reduction

from sktime.split import ExpandingGreedySplitter
from sktime.forecasting.compose import make_reduction
from sklearn.ensemble import HistGradientBoostingRegressor
from sktime.transformations.compose import Id, YtoX
from sktime.transformations.series.lag import Lag
from sktime.transformations.series.impute import Imputer
from sktime.datasets import load_airline
from sktime.split import temporal_train_test_split
from sktime.pipeline import Pipeline

from sktime.transformations.series.difference import Differencer

import warnings

In [2]:
data = pd.read_csv("../../data/monthly/ewz_stromabgabe_netzebenen_stadt_zuerich.csv",index_col=None)

data.index = pd.to_datetime(data['Timestamp'],utc=True)
data.index = data.index.tz_localize(None)  
data["y"] = data["NE7_GWh"].values
data = data.drop(columns=["Timestamp","NE5_GWh","NE7_GWh"])
data.index = pd.period_range(start=data.index[0], end= data.index[-1], freq="M")

#split data
#don't use stride (step_length=5).Keeps last splitt completely unseen. Test_size fixed by task
splitter = ExpandingGreedySplitter(test_size=12, folds=4)  # step_length=5, 
splits = list(splitter.split(data))

In [3]:
y_train = data.iloc[splits[0][0]]
y_test = data.iloc[splits[0][1]]

In [10]:
from sktime.forecasting.compose import make_reduction
from sklearn.linear_model   import Ridge
from sktime.transformations.series.difference import Differencer
from sktime.transformations.series.lag import Lag
from sktime.transformations.series.impute import Imputer
from sktime.datasets import load_airline
from sktime.split import temporal_train_test_split
from sktime.pipeline import Pipeline

y = load_airline()
y_train, y_test = temporal_train_test_split(y)

forecaster = make_reduction(Ridge(), window_length=12, strategy="recursive")

pipe = Pipeline()
pipe = pipe.add_step(Differencer(lags=[1,3]), "differencer", edges={"X": "y"})
pipe = pipe.add_step(
                    forecaster,
                    name="forecaster",
                    edges={
                        "X": "differencer",
                        "y": "y"
                    }
                    )

pipe.fit(y=y_train) #can't pass X=y_train, at predict-time this would be X=X_pred=y_pred
y_pred = pipe.predict(fh=[1, 2, 3])

C:\Users\ewzbusi\Desktop\repos\sktime\sktime\pipeline\pipeline.py:160: UserWarning: This generalised graphical pipeline is experimental, with all the usual risks of edge features. For mature alternatives, use single-purpose pipelines and compositors, such as TransformedTargetForecaster, ForecastingPipeline, ClassificationPipeline, etc., see for instance notebooks 01_forecasting.ipynb and 02_classification.ipynb athttps://github.com/sktime/sktime/blob/main/examples/.
  warnings.warn(
C:\Users\ewzbusi\Desktop\repos\sktime\sktime\pipeline\pipeline.py:160: UserWarning: This generalised graphical pipeline is experimental, with all the usual risks of edge features. For mature alternatives, use single-purpose pipelines and compositors, such as TransformedTargetForecaster, ForecastingPipeline, ClassificationPipeline, etc., see for instance notebooks 01_forecasting.ipynb and 02_classification.ipynb athttps://github.com/sktime/sktime/blob/main/examples/.
  warnings.warn(


ValueError: `X` must be passed to `predict` if `X` is given in `fit`.

In [1]:
from sktime import show_versions

show_versions()

C:\Users\ewzbusi\AppData\Roaming\Python\Python310\site-packages\statsforecast\core.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from tqdm.autonotebook import tqdm



System:
    python: 3.10.9 | packaged by Anaconda, Inc. | (main, Mar  8 2023, 10:42:25) [MSC v.1916 64 bit (AMD64)]
executable: c:\Users\ewzbusi\.conda\envs\sktime-dev\python.exe
   machine: Windows-10-10.0.19045-SP0

Python dependencies:
          pip: 23.3
       sktime: 0.25.0
      sklearn: 1.3.2
       skbase: 0.4.0
        numpy: 1.26.2
        scipy: 1.11.3
       pandas: 2.1.4
   matplotlib: 3.4.3
       joblib: 1.3.2
        numba: 0.58.1
  statsmodels: 0.14.1
     pmdarima: 2.0.3
statsforecast: 1.5.0
      tsfresh: None
      tslearn: None
        torch: 1.13.1+cpu
   tensorflow: None
tensorflow_probability: None
This is the newest local version
